## BI-FIDELTY WEIGHTED LEARNING FOR NEURAL NETWORKS 

 For details see: 
 <br>De, S., Britton, J., Reynolds, M., Skinner, R., Jansen, K., & Doostan, A. (2020). 
 <br>**"On transfer learning of neural networks using bi-fidelity data for uncertainty propagation."**
 <br>International Journal for Uncertainty Quantification, 10(6).
 <br>Link: http://www.dl.begellhouse.com/journals/52034eb04b657aea,3673619972b2eee6,3364eea04170ec7c.html
        
 To cite, please use the following BibTex entry:   
 @article{de2020transfer,
 <br>title={On transfer learning of neural networks using bi-fidelity data for uncertainty propagation},
 <br>author={De, Subhayan and Britton, Jolene and Reynolds, Matthew and Skinner, Ryan and Jansen, Kenneth and Doostan, Alireza},
 <br>journal={International Journal for Uncertainty Quantification},
 <br>volume={10},
 <br>number={6},
 <br>year={2020},
 <br>publisher={Begel House Inc.}
 <br>}

In [1]:
# import the necessary packages
import copy
import time
import numpy as np
import scipy.linalg as linalg
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RationalQuadratic, WhiteKernel, RBF, Matern
from scipy.io import loadmat  # this is the SciPy module that loads mat-files
import scipy.io 

# Load the dataset
# Coarse dataset
mat = scipy.io.loadmat('c_c.mat')
c_c=mat['c_c'] 
# Fine dataset
mat = scipy.io.loadmat('c_f.mat')
c_f=mat['c_f'] 
# Input random variables
mat = scipy.io.loadmat('rv.mat')
input_data=mat['rv']  
# Spatial average of concentrations
lf_data = np.mean(c_c,axis=1)
hf_data = np.mean(c_f,axis=1)

# For bifidelity models 
hf_train_size_bf = 20 # Number of HF datapoints used
lf_train_size_bf = 140 # Number of LF datapoints used
val_size_bf = 50 # Number of validation datapoints

# Input and Output Dimensions
dim_in = input_data.shape[1]          
dim_out = 1

# Activation Function
act_func = 'ELU'
h_size = [50, 50] # Two hidden layers with 50 neurons each

#%% --------------------------------------------------------------------------------------------------------------------
#     NN MODEL CLASS
# ----------------------------------------------------------------------------------------------------------------------

class Net(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(Net, self).__init__()
        
        # Hidden layers
        self.hidden = nn.ModuleList()
        self.hidden.append(nn.Linear(input_size, hidden_sizes[0]))
        for i in range(len(hidden_sizes)-1):
            self.hidden.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
            
        # Output layer
        self.out = nn.Linear(hidden_sizes[-1], output_size)
        
    def forward(self, x):
        for layer in self.hidden:
            if act_func == 'ReLU':
                x = F.relu(layer(x))
            elif act_func == 'tanh':
                x = F.tanh(layer(x))
            elif act_func == 'Hardtanh':
                x = F.hardtanh(layer(x))
            elif act_func == 'Sigmoid':
                x = F.sigmoid(layer(x))
            elif act_func == 'Softsign':
                x = F.softsign(layer(x))
            elif act_func == 'ELU':
                x = F.elu(layer(x))
        x = self.out(x)
        return x

In [ ]:
best_err_all = np.array([]) # saves best error for different randomization of datasets
t1_start = time.process_time() 
for iseed in range(30): # for different replication of datasets
    # For shuffling data
    np.random.seed(iseed)
    rand_inputs = np.random.permutation(input_data.shape[0])

    #%% ----------------------------------------------------------------------------------------------------------------
    #     DATA: BIFIDELITY MODELS
    # ------------------------------------------------------------------------------------------------------------------
         
    # Shuffle data    
    hf_sample_set_bf = rand_inputs[:hf_train_size_bf]
    lf_sample_set_bf = rand_inputs[hf_train_size_bf:hf_train_size_bf+lf_train_size_bf]
    val_sample_set_bf = rand_inputs[hf_train_size_bf+lf_train_size_bf:hf_train_size_bf+lf_train_size_bf+val_size_bf]
    
    # HF data
    x_bf_hfpts = input_data[hf_sample_set_bf,:]
    y_bf_train_hfdata_hfpts = hf_data[hf_sample_set_bf]
    y_bf_train_hfdata_lfpts = hf_data[lf_sample_set_bf]
    
    # LF data
    x_bf_lfpts = input_data[lf_sample_set_bf,:]
    y_bf_train_lfdata_lfpts = lf_data[lf_sample_set_bf]
    
    # Soft data
    x_bf_softpts = np.concatenate((x_bf_lfpts, x_bf_hfpts))
    y_bf_train_hfdata_softpts = np.concatenate((y_bf_train_hfdata_lfpts, y_bf_train_hfdata_hfpts))
    
    # Validation data
    x_bf_val = input_data[val_sample_set_bf,:]
    y_bf_train_val_hfdata = hf_data[val_sample_set_bf]
    
    # Write inputs as torch variables
    x_hf_train = torch.from_numpy(x_bf_hfpts).float()
    x_lfpretrain_train = torch.from_numpy(x_bf_lfpts).float()
    x_soft_train = torch.from_numpy(x_bf_softpts).float()
    x_bf_val = torch.from_numpy(x_bf_val).float()
    
    # Write targets as torch variables
    y_bf_hfpts = torch.from_numpy(y_bf_train_hfdata_hfpts).float().unsqueeze(1)
    y_bf_lfpts = torch.from_numpy(y_bf_train_lfdata_lfpts).float().unsqueeze(1)
    
    #%% ----------------------------------------------------------------------------------------------------------------
    #     BIFIDELITY MODEL: PRETRAIN STUDENT NETWORK ON LF DATA
    # ------------------------------------------------------------------------------------------------------------------
    torch.manual_seed(100) # for reproducibility 

    # LF model
    model_lfpretrain = Net(input_size=dim_in, hidden_sizes=h_size, output_size=dim_out)
    
    # Use Mean Squared Error as loss function
    criterion = nn.MSELoss(size_average=False)
    
    # Number of epochs and iterations
    batch_size = len(y_bf_lfpts)
    num_iters = len(y_bf_lfpts) / batch_size
    
    # Determine the optimizer used
    m = 'Adam'
    l_rate = 1e-4*4
    num_epochs_lfpretrain = 12000
    if m == 'SGD':
        optimizer = optim.SGD(model_lfpretrain.parameters(), lr=l_rate)    
    elif m == 'SGDMomentum':
        optimizer = optim.SGD(model_lfpretrain.parameters(), lr=l_rate, momentum=0.9)
    elif m == 'Adam':
        optimizer = optim.Adam(model_lfpretrain.parameters(), lr=l_rate, betas=(0.9, 0.99))
    elif m == 'LBFGS':
        optimizer = optim.LBFGS(model_lfpretrain.parameters(), lr=l_rate, max_iter=20, max_eval=50, 
                                tolerance_grad=1e-3, tolerance_change=1e-3, history_size=50)
        
    num_iters = int(num_iters)
    err_store = np.array([])
    
    y_pred_val = model_lfpretrain(x_bf_val).squeeze()
    y_pred_val = y_pred_val.data.numpy() 
    
    # Start timer
    start = time.time()
    
    # Copy model
    best_model = copy.deepcopy(model_lfpretrain.state_dict())
    best_err = 10000000000
    
    # Pretrain on data
    #torch.manual_seed(4)
    patience = 0 # Stopping criterion 
    data_type = 'pre-training'
    for epoch in range(num_epochs_lfpretrain):
    
        for i in range(num_iters):
            # Forward: input x and predict based on x
            xi = x_lfpretrain_train[i*batch_size:(i+1)*batch_size]
            yi = y_bf_lfpts[i*batch_size:(i+1)*batch_size]
            pred_y = model_lfpretrain(xi)
            
            # Compute loss: compare NN output to target values y_i
            loss = criterion(pred_y, yi)
            
            # Clear gradients before backward pass
            optimizer.zero_grad()
            
            # Backprop, compute gradients
            loss.backward()
            
            # Update weights
            optimizer.step() 
            
        # Predicted training and validation values
        y_pred_train = model_lfpretrain(x_lfpretrain_train)
        y_pred_train = y_pred_train.data.numpy()
        y_pred_val = model_lfpretrain(x_bf_val).squeeze()
        y_pred_val = y_pred_val.data.numpy() 
        
        divided = 10
        err_val = linalg.norm(y_pred_val - y_bf_train_val_hfdata)/linalg.norm(y_bf_train_val_hfdata)
        
        patience +=1
        
        if epoch % divided == 0:
            err_store = np.append(err_store, err_val)
        
        if err_val < best_err:
            best_err = err_val
            best_model = copy.deepcopy(model_lfpretrain.state_dict()) 
            patience = 0
            
        # Print Results
        if epoch % 6000 == 0:
            print('Type: %s, Epoch: %d, Val Err: %.5f' %(data_type,epoch, err_val))
            
        # check patience
        if patience > 8000:
            break 
    
    # Stop timer
    time_elapsed = time.time() - start
    
    # Keep the best model
    model_lfpretrain.load_state_dict(best_model)
    torch.save(model_lfpretrain.state_dict(), 'battery_lf_pretraining_bf_model.pt')
    
    #%% ----------------------------------------------------------------------------------------------------------------
    #     BIFIDELITY MODEL
    #     TRAIN THE TEACHER WITH HF DATA
    # ------------------------------------------------------------------------------------------------------------------
    batch_size = 1
    num_epochs_fwl = 1000
    # Kernel function 
    kernel = 1.0 * RationalQuadratic(length_scale_bounds=(20,21)) + WhiteKernel(noise_level_bounds=(1e-7,1e-5)) 
    
    # train on strong dataset
    gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10).fit(x_hf_train, y_bf_hfpts)
    y_gp_hf_mean, y_gp_hf_cov = gp.predict(x_hf_train, return_cov=True)
    y_gp_hf_mean = y_gp_hf_mean.squeeze()
        
    # Get mean m and covariance matrix K for soft data
    y_gp_soft_mean, y_gp_soft_cov = gp.predict(x_soft_train, return_cov=True)
    y_gp_soft_mean = y_gp_soft_mean.squeeze()  
    
    # Teacher T(x_t) = g(m(x_t)), here g = identity
    # \bar{y}(x_t) = T(x_t)
    y_pred_gp_train = y_gp_soft_mean
    y_pred_gp_train = torch.from_numpy(y_pred_gp_train).float().reshape(-1,1)
    
    # Uncertainty Sigma(x_t) = h(K(x_t, x_t)), here h = I
    sigma_soft_train = np.diag(y_gp_soft_cov)
    
    # Val error
    err_gp_val = linalg.norm(y_gp_soft_mean - y_bf_train_hfdata_softpts)/linalg.norm(y_bf_train_hfdata_softpts)
    
    #%% ----------------------------------------------------------------------------------------------------------------
    #     FINE-TUNE THE STUDENT NETWORK WITH SOFT DATA
    # ------------------------------------------------------------------------------------------------------------------
    err_fwl_train = np.zeros((len(betas), 1))
    err_fwl_val = np.zeros((len(betas), 1))
    y_pred_fwl_train = np.zeros((len(betas), len(y_bf_train_hfdata_softpts)))
    y_pred_fwl_val = np.zeros((len(betas), len(y_bf_train_val_hfdata)))
    time_fwl = np.zeros((len(betas), 1))
    best_err_fwl = np.zeros((len(betas), 1))
    err_store_fwl = np.zeros((len(betas), 
                              int(np.floor(num_epochs_fwl/divided))))
    
    beta = 0.01
    
    # Compute eta_2
    eta = np.exp(-beta * sigma_soft_train) # why? aggregation
    
    # Load model with LF trained weight to use with HF data
    model_fwl = Net(input_size=dim_in, hidden_sizes=h_size, output_size=dim_out)
    model_fwl.load_state_dict(torch.load('battery_lf_pretraining_bf_model.pt'))
    
    # Use Mean Squared Error as loss function
    criterion = nn.MSELoss(size_average=False)
    num_iters = len(y_pred_gp_train)
    # Determine the optimizer used
    m = 'Adam'
    l_rate = 1e-4 # learning rate
    if m == 'SGD':
        optimizer = optim.SGD(model_fwl.parameters(), lr=l_rate) 
    elif m == 'SGDMomentum':
        optimizer = optim.SGD(model_fwl.parameters(), lr=l_rate, momentum=0.9, nesterov=True)
    elif m == 'Adam':
        optimizer = optim.Adam(model_fwl.parameters(), lr=l_rate, betas=(0.9, 0.99))
    elif m == 'LBFGS':
        optimizer = optim.LBFGS(model_fwl.parameters(), lr=l_rate, max_iter=200)
        
    data_type = 'soft_finetuning_bf'
    best_err_fwl=10000
    err_store_fwl = np.array([])
    patience = 0
    for epoch in range(num_epochs_fwl):
    
        for i in range(num_iters):
            # Forward: input x and predict based on x
            xi = x_soft_train[i*batch_size:(i+1)*batch_size]
            yi = y_pred_gp_train[i*batch_size:(i+1)*batch_size]
            pred_y = model_fwl(xi)                
            
            # Compute loss: compare NN output to target values y_i
            loss = criterion(pred_y, yi)
            
            # Clear gradients before backward pass
            optimizer.zero_grad()
            
            # Backprop, compute gradients
            loss.backward()
            
            # Update weights
            optimizer.param_groups[0]['lr'] *= eta[i]
            # perform one step of optimization
            optimizer.step() 
            # change the learning rate back to its original value
            optimizer.param_groups[0]['lr'] *= (1.0/eta[i])
            
        # Predicted training and validation values
        y_pred_train = model_fwl(x_soft_train)
        y_pred_train = y_pred_train.data.numpy()
        y_pred_val = model_fwl(x_bf_val).squeeze()
        y_pred_val = y_pred_val.data.numpy() 
        
        divided = 10
        err_val = linalg.norm(y_pred_val - y_bf_train_val_hfdata)/linalg.norm(y_bf_train_val_hfdata)
        
        patience += 1
        
        if epoch % divided == 0:
            err_store_fwl = np.append(err_store_fwl, err_val)
        
        if err_val < best_err_fwl:
            best_err_fwl = err_val
            best_model = copy.deepcopy(model_fwl.state_dict())
            patience = 0 
            
        # Print Results
        if epoch % 500 == 0:
            print('Type: %s, Epoch: %d, Val Err: %.5f' %(data_type,epoch, err_val))
        
        # check patience
        if patience > 500:
            break
    
    # Stop timer
    time_elapsed = time.time() - start
    
    # Keep the best model
    model_fwl.load_state_dict(best_model)
    torch.save(model_fwl.state_dict(), 'battery_fwl_bf_model.pt')
    
    best_err_all = np.append(best_err_all,np.around(best_err_fwl,6))
    
t1_elapsed = time.process_time() - t1_start

Type: pre-training, Epoch: 0, Val Err: 1.00004
Type: pre-training, Epoch: 6000, Val Err: 0.01189


C:\Users\subha\.conda\envs\pytorchenv\lib\site-packages\torch\nn\_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Type: soft_finetuning_bf, Epoch: 0, Val Err: 0.00555
Type: soft_finetuning_bf, Epoch: 500, Val Err: 0.00613
Type: pre-training, Epoch: 0, Val Err: 1.00005
Type: pre-training, Epoch: 6000, Val Err: 0.01202


C:\Users\subha\.conda\envs\pytorchenv\lib\site-packages\torch\nn\_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Type: soft_finetuning_bf, Epoch: 0, Val Err: 0.01101
Type: soft_finetuning_bf, Epoch: 500, Val Err: 0.01028


In [10]:
best_err_all

array([0.006283, 0.010089, 0.011663, 0.012303])